In [102]:

from sklearn.metrics import accuracy_score, classification_report

from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference
from sklearn.metrics import confusion_matrix

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from fairlearn.metrics import demographic_parity_difference
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

from fairlearn.metrics import (
    equalized_odds_difference,    
)




In [ ]:
df = pd.read_csv('clean_data.csv', encoding='utf-8')

data = df.sample(n=100000, random_state=1)

X = data.drop(['ANZSOC Division','Date'], axis=1)
X = pd.get_dummies(X)  # Convert categorical variables to numerical format
Y = data['ANZSOC Division']

X.head()
##上面这些都没用的不用看


In [ ]:
df = pd.read_csv('clean_data.csv', encoding='utf-8')

data = df.sample(n=100000, random_state=1)

data = data.drop(columns=['Date'])

# Label encoding for categorical variables
label_encoders = {}
for column in ['Police District', 'Police Area', 'Person/Organisation', 'Age Group', 'ANZSOC Division','Ethnic Group']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Train-test split
X = data.drop('ANZSOC Division', axis=1)
Y = data['ANZSOC Division']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
base_classifier = RandomForestClassifier(random_state=42)
base_classifier.fit(X_train, Y_train)

# Prediction and evaluation
y_pred = base_classifier.predict(X_test)
print(f"Accuracy: {accuracy_score(Y_test, y_pred)}")
print(f"Classification Report: \n{classification_report(Y_test, y_pred)}")


In [111]:
# Evaluate fairness-unaware model using Fairlearn's demographic_parity_difference
sensitive_feature_train = X_train['Ethnic Group']
y_pred_train = base_classifier.predict(X_train)

unfairness = demographic_parity_difference(Y_train, y_pred_train, sensitive_features=sensitive_feature_train)
print(f"Fairness-unaware model demographic parity difference: {unfairness}")


Fairness-unaware model demographic parity difference: 0.7768473367895267


In [112]:
def calculate_disparate_impact(y_true, y_pred, sensitive_features):
    # Compute the probabilities of a positive outcome for each group
    group_labels = list(set(sensitive_features))
    outcomes = {label: [] for label in group_labels}
    
    for true, pred, feature in zip(y_true, y_pred, sensitive_features):
        outcomes[feature].append((true == pred))
    
    probabilities = {label: sum(values) / len(values) for label, values in outcomes.items()}
    
    # Assuming label '0' is for the privileged group and '1' is for the unprivileged group
    # Adjust these based on your actual labels
    try:
        return probabilities[1] / probabilities[0]
    except ZeroDivisionError:
        return float('inf')  # Return infinity if the denominator is zero

In [114]:
di_ratio = calculate_disparate_impact(Y_train, y_pred_train_base, sensitive_feature_train)
print(f"Disparate Impact Ratio (Unaware Model): {di_ratio}")

Disparate Impact Ratio (Unaware Model): 0.8241243423526932
